In [104]:
import pandas as pd
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from torch.nn.utils.rnn import pad_sequence
from torch.optim import AdamW

In [105]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Идея

Судя по тому, что в задании выделена фраза "по эмбеддингу пользователя искать похожие эмбеддинги фильмов", задача сводится к построению таких эмбеддингов, чтобы косинусное сходство между эмбеддингами пользователей и фильмов, которые им нравятся, было максимальным.

Для фильмов нам даны их жанры, теги, проставленные пользователями, а также другой набор тегов с коэффициентами их релевантности для каждого фильма. Всю эту информацию можно использовать для построения эмбеддингов фильмов.
Для пользователей нам даны их индексы, проставленные теги и оценки. По индексам можно строить эмбеддингов, по оценкам - считать ошибку, а с тегами сложнее. Пользователь может ставить теги совершенно разным фильмам, в том числе и тем, которые ему не понравились. Поэтому проставленные пользователем теги не дадут никакой информации о его предпочтениях.

В итоге получаем следующий пайплайн. Строим эмбеддинги фильмов по всей информации, которая у нас о них есть, строим эмбеддинги по пользовательским индексам, вычисляем косинусное сходство между полученными эмбеддингами, и считаем ошибку, используя косинусное сходство и поставленную оценку.

## Предобработка

In [106]:
genome_scores = pd.read_csv("ml-latest/genome-scores.csv")
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02900
1,1,2,0.02375
2,1,3,0.05425
3,1,4,0.06875
4,1,5,0.16000


In [107]:
genome_tags = pd.read_csv("ml-latest/genome-tags.csv")
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


Сразу обработаем фиксированный набор тегов, для которых посчитана релевантность. Так как их индексы идут подряд, сохраним их в отдельный тензор без изменений.

In [108]:
other_tags = torch.Tensor([i for i in range(genome_tags.shape[0])]).long().to(device)

In [109]:
tags = pd.read_csv("ml-latest/tags.csv")
movies = pd.read_csv("ml-latest/movies.csv", index_col="movieId")
ratings = pd.read_csv("ml-latest/ratings.csv")

Для обработки пользовательский тегов проведем лемматизацию и создадим словарь индексов. Так как большинство тегов однословны, нет смысла проводить какую либо токенизацию или использовать предобученную токенизацию.

In [110]:
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


Удалим пропущенные теги

In [111]:
tags.isna().sum()

userId        0
movieId       0
tag          16
timestamp     0
dtype: int64

In [112]:
tags.dropna(inplace=True)

In [113]:
tags.isna().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

In [114]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1108981 entries, 0 to 1108996
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1108981 non-null  int64 
 1   movieId    1108981 non-null  int64 
 2   tag        1108981 non-null  object
 3   timestamp  1108981 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 42.3+ MB


Переводим колонку тегов из типа object в строки, приводим к нижнему регистру и применяем токенизацию. Затем создаём словарь тегов и сопоставляем с индексами. Оставим нулевой индекс для случаев, когда тег к фильму будет отсутствовать.

In [115]:
from nltk.stem.snowball import SnowballStemmer

In [116]:
stemmer = SnowballStemmer(language="english")
tags["tag"] = tags["tag"].astype(str).apply(lambda x: stemmer.stem(x.lower()))

In [117]:
unique_tags = tags["tag"].unique()
tags_to_idx = {t: i+1 for i, t in enumerate(unique_tags)}
tags["tag"] = tags["tag"].map(tags_to_idx)
tags.head()

,userId,movieId,tag,timestamp
0,14,110,1,1443148538
1,14,110,2,1443148532
2,14,260,3,1442169410
3,14,260,4,1442169421
4,14,318,5,1442615195


Сохраним результат в pd.Series, где каждому индексу фильма будет соответстовать набор индексов пользовательских тегов.

In [118]:
tags_concatenated = tags.groupby("movieId")["tag"].agg(list)

In [119]:
tags_concatenated.head()

movieId
1    [91, 1008, 1009, 1010, 60, 1011, 496, 873, 206...
2    [275, 1138, 91, 1724, 783, 4153, 4154, 503, 41...
3    [9981, 5216, 10437, 7155, 5410, 4380, 655, 104...
4    [2953, 13305, 2953, 443, 15740, 443, 1243, 243...
5    [4638, 4638, 1238, 1405, 1323, 2167, 22148, 27...
Name: tag, dtype: object

У фильмов может быть несколько одинаковых тегов, но не будем это исправлять. Это поможет получить более точное представление фильмов.

Полученные наборы индексов тегов добавим в датафрейм с информацией про фильмы

In [120]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [121]:
movies["genres"] = movies["genres"].apply(lambda x: x.split("|")) #переводим жанры из строк в списки

In [122]:
movies["tags"] = tags_concatenated

In [123]:
movies.isna().sum()

title         0
genres        0
tags      12117
dtype: int64

Там, где теги отсутствуют, проставляем нулевые индексы

In [124]:
movies["tags"] = movies["tags"].apply(lambda x: [0] if not isinstance(x, list) else x)

In [125]:
movies.head()

,title,genres,tags
movieId,,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[91, 1008, 1009, 1010, 60, 1011, 496, 873, 206..."
2,Jumanji (1995),"[Adventure, Children, Fantasy]","[275, 1138, 91, 1724, 783, 4153, 4154, 503, 41..."
3,Grumpier Old Men (1995),"[Comedy, Romance]","[9981, 5216, 10437, 7155, 5410, 4380, 655, 104..."
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]","[2953, 13305, 2953, 443, 15740, 443, 1243, 243..."
5,Father of the Bride Part II (1995),[Comedy],"[4638, 4638, 1238, 1405, 1323, 2167, 22148, 27..."


## Разделение выборки

Разделим выборку, содержащую индексы пользователей и проставленные ими рейтинги на 3 части - обучающую, валидационную и тестовую в пропорции 7:2:1. Так как нам нужно создать эмбеддинги для всех пользователей, то все индексы пользователей должны присутствовать в каждой из выборок. Поэтому сгруппируем датафрейм по их индексами и разделение будем проводить внутри полученных групп. Для того, чтобы приблизиться к реальной ситуации, отсортируем строки внутри каждой из групп по времени, старые данные занесем в обучающую выборку, а новые - в валидационную и тестовую 

In [126]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [127]:
#функция для отделения доли выборки в каждой группе
def train_test_split(group, train_size):
    threshold = int(len(group) * train_size) #определяем пороговое значение для разделения группы
    return group.iloc[:threshold]

In [128]:
ratings.sort_values(["userId", "timestamp"], inplace=True) #сортируем сначала по пользователям, затем по времени
                                                           #в порядке возрастания значений
train_ratings = ratings.groupby("userId").apply(train_test_split, 0.7).reset_index(drop=True) #отделяем тренировочные данные
val_test_ratings = pd.concat([ratings, train_ratings]).drop_duplicates(keep=False).reset_index(drop=True) #остаток заносим
                                                                                            #в валидационную и тестовую

In [129]:
train_ratings.head()

,userId,movieId,rating,timestamp
0,1,3826,2.0,1256677210
1,1,307,3.5,1256677221
2,1,1590,2.5,1256677236
3,1,2478,4.0,1256677239
4,1,3698,3.5,1256677243


In [130]:
#аналогично, отделяем валидационную выборку от тестовой
val_ratings = val_test_ratings.groupby("userId").apply(train_test_split, 2/3).reset_index(drop=True)
test_ratings = pd.concat([val_test_ratings, val_ratings]).drop_duplicates(keep=False).reset_index(drop=True)

In [131]:
val_ratings.head()

,userId,movieId,rating,timestamp
0,1,1091,1.5,1256677471
1,1,1591,1.5,1256677475
2,1,3893,3.5,1256677486
3,2,3363,4.0,1192913596
4,2,2707,3.5,1192913600


In [132]:
test_ratings.head()

,userId,movieId,rating,timestamp
0,1,2986,2.5,1256677496
1,1,2840,3.0,1256677500
2,2,1296,4.5,1192913608
3,2,1186,3.5,1192913611
4,3,2024,3.0,945141611


In [133]:
train_ratings.shape[0], val_ratings.shape[0], test_ratings.shape[0]

(19299130, 5548657, 2905657)

In [134]:
ratings.shape

(27753444, 4)

Получили нужные пропорции

При инициализации также закодируем жанры фильмов. Для жанров не требуется предварительной обработки, так как их мало и они все разные. Их перечень есть в документации датасета, выпишем его сюда и создадим словарь

In [135]:
genres = [
    "(no genres listed)",
    "Action", 
    "Adventure", 
    "Animation", 
    "Children", 
    "Comedy", 
    "Crime", 
    "Documentary", 
    "Drama", 
    "Fantasy", 
    "Film-Noir", 
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
    "IMAX"
]
genres_to_idx = {genre: index for index, genre in enumerate(genres)}

## Создание датасета

Внутри батча будут содержаться все описанные выше признаки. Создавать батч будем следующим образом: по полученному индексу обращаемся к таблице с пользователями, фильмами и оценками, из нее получаем индекс фильма и используем его для получения всех остальных признаков во второй таблице.

Так как мы оперерируем тензорами, то внутри батча будем добавлять паддинги из нулей. Нули как раз подходят, так как до этого мы обозначали ими отсутствие тегов или неуказанный жанр.

In [136]:
class MovieDataset(Dataset):
    def __init__(self, movies_data, user_data, relevance_data):
        self.user_idx = user_data["userId"]
        self.ratings = user_data["rating"]
        self.movie_idx = user_data["movieId"]
        self.tag_idx = movies_data["tags"]
        self.genre_idx = movies_data["genres"].apply(lambda x: [genres_to_idx[genre] for genre in x]) #кодируем жанры по словарю
        self.relevances = relevance_data
    
    def __getitem__(self, index):
        user_id = self.user_idx[index]
        rating = self.ratings[index]
        movie_id = self.movie_idx[index]
        tags = torch.Tensor(self.tag_idx[movie_id]).int()
        genres = torch.Tensor(self.genre_idx[movie_id]).int()
        relevance = torch.Tensor(self.relevances[self.relevances["movieId"] == movie_id]["relevance"].values)
        return tags, genres, user_id, rating, relevance, index
    
    def __len__(self):
        return self.user_idx.shape[0]
    
    def collate_function(self, batch):
        tags, genres, user_id, rating, relevance, index = zip(*batch)
        batched_tags = pad_sequence(tags, batch_first=True, padding_value=0)
        batched_genres = pad_sequence(genres, batch_first=True, padding_value=0)
        user_id = torch.Tensor(user_id).int()
        rating = torch.Tensor(rating).int()
        relevance = pad_sequence(relevance, batch_first=True, padding_value=0.0)
        return {
            "user_id": user_id, 
            "tags": batched_tags,
            "genres": batched_genres, 
            "rating": rating, 
            "relevance": relevance,
            "index": index}

Создаем даталоадеры, объекты перемешиваем с фиксированной случайностью

In [137]:
torch.manual_seed(42)
batch_size = 64

train_dataset = MovieDataset(movies, train_ratings, genome_scores)
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    collate_fn=train_dataset.collate_function,
    shuffle=True,
)

val_dataset = MovieDataset(movies, val_ratings, genome_scores)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=val_dataset.collate_function,
    shuffle=True,
)

test_dataset = MovieDataset(movies, test_ratings, genome_scores)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=val_dataset.collate_function,
    shuffle=True,
)

## Обучение

Как было описано в начале, наша модель будет создавать эмбеддинги фильмов по их признакам и эмбеддинги пользователям по индексам. Затем мы будем считать косинусное сходство между двумя полученными эмбеддингами и на его основе вычислять лосс, о котором будет написано дальше.

Для получения эмбеддингов фильмов воспользуемся тремя простыми nn.Embedding(), каждый из которых будет создавать векторные представления пользовательских тегов, жанров и фиксированных тегов соответственно. Далее полученные эмбеддинги сконкатенируем, пропустим через линейный слой, чтобы привести к размерности эмбеддинга пользователя, и вычислим лосс. Эмбеддинги пользователя также будем строить с помощью nn.Embedding().

Можно было попробовать закодировать признаки и по-другому - например, с помощью предобученной модели BERT или обучить свой TransformerEncoder. Но предобученная модель кажется слишком избыточной для такой задачи, так как нам особо не нужны точные смысловые представления тегов и жанров. А трансформерные архитектуры все-таки больше подходят для более длинных последовательностей, и тоже могли оказаться избытычными. В нашем случае наши признаки однословные и ни теги, ни жанры друг с другом не так сильно связаны, чтобы образовывать какой-то контекст.

In [138]:
class MovieModel(nn.Module):
    def __init__(self, tag_vocab_size, genre_vocab_size, user_vocab_size, embedding_dim, other_tags, batch_size):
        super(MovieModel, self).__init__()
        
        self.other_tags = other_tags.repeat(batch_size, 1) #набор фиксированных тегов, соответствующий числу сэмплов в батче
        
        self.tag_embedder = nn.Embedding(tag_vocab_size, embedding_dim, padding_idx=0) #эмбеддер пользовательских тегов
        self.genre_embedder = nn.Embedding(genre_vocab_size, embedding_dim, padding_idx=0) #эмбеддер жанров
        self.user_embedder = nn.Embedding(user_vocab_size, embedding_dim) #эмбеддер пользовательских индексов
        self.other_tag_embedder = nn.Embedding(len(other_tags), embedding_dim, padding_idx=0) #эмбеддер фиксированных тегов
        
        self.fc = nn.Linear(3 * embedding_dim, embedding_dim)
        
    def forward(self, input_batch):
        tag_emb = torch.mean(self.tag_embedder(input_batch["tags"]), dim=1)
        genre_emb = torch.mean(self.genre_embedder(input_batch["genres"]), dim=1)
        other_tag_emb = torch.mean(
            input_batch["relevance"].unsqueeze(-1) * self.other_tag_embedder(self.other_tags), dim=1
        ) #здесь мы умножаем каждое представление фиксированных тегов на соответствующие коэффициенты релевантности
        
        tag_genre_emb = torch.cat((tag_emb, genre_emb, other_tag_emb), dim=1) #конкатенируем полученные эмбеддинги
        movie_emb = self.fc(tag_genre_emb) #получаем эмбеддинг фильма
        
        user_emb = self.user_embedder(input_batch["user_id"])
        
        return user_emb, movie_emb

Чтобы получить лосс, сначала посчитаем косинусное сходство между векторами, затем переведем его в диапазон от 0 до 5. Затем вычислим сам лосс - Root mean squared error между оценкой пользователя и полученным значением сходства. Я выбрал RMSE, так как его среднее значение можно использовать в качестве интерпретируемой метрики на валидации - будет понятно, на сколько баллов наша предсказанная оценка будет отклоняться от истинной в среднем.

In [139]:
def compute_loss(user_emb, movie_emb, rating):
    cos_sim = nn.CosineSimilarity(dim=1)
    
    sim_norm = (cos_sim(user_emb, movie_emb) + 1) * 5 #переводим из диапазона [-1, 1] в [0, 5]
    rmse_loss = torch.sqrt(torch.mean((sim_norm - rating) ** 2))
    
    return rmse_loss

In [ ]:
embedding_dim = 100
lr = 1e-5
epochs = 5

model = MovieModel(
    unique_tags.shape[0]+1, 
    len(genres), 
    ratings["userId"].unique().shape[0]+1, 
    embedding_dim, 
    other_tags, 
    batch_size
)
model.to(device)

optimizer = AdamW(model.parameters(), lr)

pbar = tqdm(total=len(train_dataloader)*epochs)

num_iter = 1
val_every = 1000 #каждые val_every итераций будем проводить валидацию

for epoch in range(epochs):
    for batch in train_dataloader:
        model.train()

        for key, val in batch.items(): #переводим все тензоры на устройство вычислений
            if key != "index":
                batch[key] = val.to(device)

        optimizer.zero_grad()

        user_emb, movie_emb = model(batch)
        loss = compute_loss(user_emb, movie_emb, batch["rating"])

        loss.backward()
        optimizer.step()

        pbar.desc = f"Epoch {epoch+1}/{epochs}. Train loss: {round(loss.item(), 3)}"
        pbar.update()
        
        num_iter += 1
        
        if num_iter % val_every == 0:
            model.eval()
            val_loss = 0
            
            for batch in tqdm(val_dataloader):
                for key, val in batch.items():
                    if key != "index":
                        batch[key] = val.to(device)
                user_emb, movie_emb = model(batch)
                val_loss += compute_loss(user_emb, movie_emb, batch["rating"])
                
            print(f"Validation loss: {round(val_loss.item()/len(val_dataloader), 3)}")
                
        
pbar.close()

К сожалению, на полном датасете модель будет обучаться слишком долго. Но чтобы показать, что она рабочая и получить какие-то результаты, я приложил файл SMALL_Movie_recommendations.ipynb, где модель работает на коротком датасете. Мне пришлось сделать это в отдельном файле, так как для маленького датасета нет таблиц genome_score и genome_tags, поэтому код нужно было немного изменить. Пожалуйста, посмотрите второй ноутбук для ознакомления с полученными результатами.